# Preparing the MIMIC 3 dataset

MIMIC3 can be downloaded from [here](https://mimic.physionet.org/gettingstarted/access/).

We use the same preprocessing as the authors of the GRU-ODE-Bayer paper and provide the final data preparation notebook that is missing in the original repository. First, execute the notebooks *admissions*, *outputs*, *labevents* and *prescriptions* provided [here](https://github.com/edebrouwer/gru_ode_bayes/tree/master/data_preproc/MIMIC).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
import numpy as np

In [2]:
file_path="./files/mimiciii/1.4/"
outfile_path="../mimic/"
lab_df=pd.read_csv(file_path+"LAB_processed.csv")[["SUBJECT_ID","HADM_ID","CHARTTIME","VALUENUM","LABEL"]]
inputs_df=pd.read_csv(file_path+"INPUTS_processed.csv")[["SUBJECT_ID","HADM_ID","CHARTTIME","AMOUNT","LABEL"]]
outputs_df=pd.read_csv(file_path+"OUTPUTS_processed.csv")[["SUBJECT_ID","HADM_ID","CHARTTIME","VALUE","LABEL"]]
presc_df=pd.read_csv(file_path+"PRESCRIPTIONS_processed.csv")[["SUBJECT_ID","HADM_ID","CHARTTIME","DOSE_VAL_RX","DRUG"]]

/tmp/ipykernel_108364/1527329938.py:3: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  lab_df=pd.read_csv(file_path+"LAB_processed.csv")[["SUBJECT_ID","HADM_ID","CHARTTIME","VALUENUM","LABEL"]]
/tmp/ipykernel_108364/1527329938.py:4: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  inputs_df=pd.read_csv(file_path+"INPUTS_processed.csv")[["SUBJECT_ID","HADM_ID","CHARTTIME","AMOUNT","LABEL"]]
/tmp/ipykernel_108364/1527329938.py:6: DtypeWarning: Columns (12,17) have mixed types. Specify dtype option on import or set low_memory=False.
  presc_df=pd.read_csv(file_path+"PRESCRIPTIONS_processed.csv")[["SUBJECT_ID","HADM_ID","CHARTTIME","DOSE_VAL_RX","DRUG"]]


In [3]:
inputs_df["VALUENUM"]=inputs_df["AMOUNT"]
inputs_df.head()
inputs_df=inputs_df.drop(columns=["AMOUNT"]).copy()

outputs_df["VALUENUM"]=outputs_df["VALUE"]
outputs_df=outputs_df.drop(columns=["VALUE"]).copy()

presc_df["VALUENUM"]=presc_df["DOSE_VAL_RX"]
presc_df=presc_df.drop(columns=["DOSE_VAL_RX"]).copy()
presc_df["LABEL"]=presc_df["DRUG"]
presc_df=presc_df.drop(columns=["DRUG"]).copy()

inputs_df["Origin"]="Inputs"
lab_df["Origin"]="Lab"
outputs_df["Origin"]="Outputs"
presc_df["Origin"]="Prescriptions"

merged_df1=(inputs_df.append(lab_df)).reset_index()
merged_df2=(merged_df1.append(outputs_df)).reset_index()
merged_df2.drop(columns="level_0",inplace=True)
merged_df=(merged_df2.append(presc_df)).reset_index()

assert(merged_df["LABEL"].nunique()==(inputs_df["LABEL"].nunique()+lab_df["LABEL"].nunique()+outputs_df["LABEL"].nunique()+presc_df["LABEL"].nunique()))

/tmp/ipykernel_108364/1618795919.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df1=(inputs_df.append(lab_df)).reset_index()
/tmp/ipykernel_108364/1618795919.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df2=(merged_df1.append(outputs_df)).reset_index()
/tmp/ipykernel_108364/1618795919.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df=(merged_df2.append(presc_df)).reset_index()


In [4]:
merged_df.head()

,level_0,index,SUBJECT_ID,HADM_ID,CHARTTIME,LABEL,VALUENUM,Origin
0,0,0.0,27063,139787.0,2133-02-05 06:29:00,Potassium Chloride,1.354906,Inputs
1,1,1.0,27063,139787.0,2133-02-05 06:59:00,Potassium Chloride,1.354906,Inputs
2,2,2.0,27063,139787.0,2133-02-05 07:29:00,Potassium Chloride,1.354906,Inputs
3,3,3.0,27063,139787.0,2133-02-05 07:59:00,Potassium Chloride,1.354906,Inputs
4,4,4.0,27063,139787.0,2133-02-05 08:29:00,Potassium Chloride,1.354906,Inputs


In [5]:
merged_df['CHARTTIME']=pd.to_datetime(merged_df["CHARTTIME"], format='%Y-%m-%d %H:%M:%S')
ref_time=merged_df.groupby("HADM_ID")["CHARTTIME"].min()

merged_df_1=pd.merge(ref_time.to_frame(name="REF_TIME"),merged_df,left_index=True,right_on="HADM_ID")
merged_df_1["TIME_STAMP"]=merged_df_1["CHARTTIME"]-merged_df_1["REF_TIME"]
assert(len(merged_df_1.loc[merged_df_1["TIME_STAMP"]<timedelta(hours=0)].index)==0)

In [6]:
label_dict=dict(zip(list(merged_df_1["LABEL"].unique()),range(len(list(merged_df_1["LABEL"].unique())))))
merged_df_1["LABEL_CODE"]=merged_df_1["LABEL"].map(label_dict)

merged_df_short=merged_df_1[["HADM_ID","VALUENUM","TIME_STAMP","LABEL_CODE","Origin"]]

In [7]:
label_dict_df=pd.Series(merged_df_1["LABEL"].unique()).reset_index()
label_dict_df.columns=["index","LABEL"]
label_dict_df["LABEL_CODE"]=label_dict_df["LABEL"].map(label_dict)
label_dict_df.drop(columns=["index"],inplace=True)
label_dict_df.to_csv(outfile_path+"label_dict.csv")

In [8]:
merged_df_short["VALUENUM"] = merged_df_short["VALUENUM"].astype(float)

/tmp/ipykernel_108364/963878085.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_short["VALUENUM"] = merged_df_short["VALUENUM"].astype(float)


In [9]:
merged_df_short["TIME_STAMP"].dt.total_seconds()/60

4808862        0.0
4808863      791.0
4808864     1770.0
4808892     3420.0
4808893     3875.0
             ...  
13250419     255.0
13250420     255.0
13250421    1695.0
13250422    3135.0
13250423    3135.0
Name: TIME_STAMP, Length: 13420539, dtype: float64

In [10]:
merged_df_short=merged_df_short.loc[(merged_df_short["TIME_STAMP"].dt.total_seconds()/60<2880)]
print("Number of patients considered :"+str(merged_df_short["HADM_ID"].nunique()))

Number of patients considered :24681


In [11]:
merged_df_short.drop(["Origin"], axis=1, inplace=True)
merged_df_short

,HADM_ID,VALUENUM,TIME_STAMP,LABEL_CODE
4808862,100007.0,15.00,0 days 00:00:00,0
4808863,100007.0,12.00,0 days 13:11:00,0
4808864,100007.0,10.00,1 days 05:30:00,0
5075112,100007.0,25.00,0 days 00:00:00,1
5075113,100007.0,25.00,0 days 13:11:00,1
...,...,...,...,...
12293315,199999.0,5.00,1 days 18:00:00,18
12293920,199999.0,7.47,1 days 18:47:00,33
13250419,199999.0,3.00,0 days 04:15:00,38
13250420,199999.0,81.00,0 days 04:15:00,67


In [12]:
complete_df = merged_df_short
labels = complete_df["LABEL_CODE"].unique()
value_columns = []
mask_columns  = []
for num in labels:
    name = "Value_label_" + str(num)
    name2 = "Mask_label_" + str(num)
    value_columns.append(name)
    mask_columns.append(name2)
    complete_df[name] = 0
    complete_df[name2] = 0
    complete_df[name] = complete_df[name].astype(float)

/tmp/ipykernel_108364/4194740534.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_df[name2] = 0
/tmp/ipykernel_108364/4194740534.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_df[name] = 0
/tmp/ipykernel_108364/4194740534.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


In [13]:
complete_df.iloc[0]['LABEL_CODE']

0

In [14]:
from tqdm import tqdm
complete_df.dropna(inplace=True)

for index, row in tqdm(complete_df.iterrows()):
    # print(row)
    # print(type(row["LABEL_CODE"]))
    name = "Value_label_" + str(int(row["LABEL_CODE"]))
    name2 = "Mask_label_" + str(int(row["LABEL_CODE"]))
    complete_df.at[index, name] = row["VALUENUM"]
    complete_df.at[index, name2] = 1

3857549it [04:44, 13582.42it/s]


In [15]:
complete_df.drop(["VALUENUM", "LABEL_CODE"], axis=1, inplace=True)
complete_df = complete_df.groupby(["HADM_ID", "TIME_STAMP"], as_index=False).max()
for x in mask_columns:
    assert(len(complete_df.loc[complete_df[x]>1])==0)
complete_df

/tmp/ipykernel_108364/3134311965.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_df = complete_df.groupby(["HADM_ID", "TIME_STAMP"], as_index=False).max()
/tmp/ipykernel_108364/3134311965.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_df = complete_df.groupby(["HADM_ID", "TIME_STAMP"], as_index=False).max()


,HADM_ID,TIME_STAMP,Value_label_0,Mask_label_0,Value_label_1,Mask_label_1,Value_label_2,Mask_label_2,Value_label_3,Mask_label_3,...,Value_label_92,Mask_label_92,Value_label_95,Mask_label_95,Value_label_94,Mask_label_94,Value_label_93,Mask_label_93,Value_label_91,Mask_label_91
0,100007.0,0 days 00:00:00,15.0,1,25.0,1,0.0,0,103.0,1,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1,100007.0,0 days 02:20:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
2,100007.0,0 days 02:45:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
3,100007.0,0 days 03:29:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
4,100007.0,0 days 13:11:00,12.0,1,25.0,1,8.9,1,107.0,1,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129396,199999.0,1 days 09:55:00,13.0,1,23.0,1,7.9,1,101.0,1,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1129397,199999.0,1 days 18:00:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1129398,199999.0,1 days 18:26:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1129399,199999.0,1 days 18:44:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0


In [16]:
complete_df = complete_df.groupby(["HADM_ID", "TIME_STAMP"], as_index=False).max()
complete_df

/tmp/ipykernel_108364/425130907.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_df = complete_df.groupby(["HADM_ID", "TIME_STAMP"], as_index=False).max()
/tmp/ipykernel_108364/425130907.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_df = complete_df.groupby(["HADM_ID", "TIME_STAMP"], as_index=False).max()


,HADM_ID,TIME_STAMP,Value_label_0,Mask_label_0,Value_label_1,Mask_label_1,Value_label_2,Mask_label_2,Value_label_3,Mask_label_3,...,Value_label_92,Mask_label_92,Value_label_95,Mask_label_95,Value_label_94,Mask_label_94,Value_label_93,Mask_label_93,Value_label_91,Mask_label_91
0,100007.0,0 days 00:00:00,15.0,1,25.0,1,0.0,0,103.0,1,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1,100007.0,0 days 02:20:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
2,100007.0,0 days 02:45:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
3,100007.0,0 days 03:29:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
4,100007.0,0 days 13:11:00,12.0,1,25.0,1,8.9,1,107.0,1,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129396,199999.0,1 days 09:55:00,13.0,1,23.0,1,7.9,1,101.0,1,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1129397,199999.0,1 days 18:00:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1129398,199999.0,1 days 18:26:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1129399,199999.0,1 days 18:44:00,0.0,0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0


In [17]:
unique_ids=np.arange(complete_df["HADM_ID"].nunique())
np.random.shuffle(unique_ids)
d=dict(zip(complete_df["HADM_ID"].unique(),unique_ids))

Unique_id_dict=pd.Series(complete_df["HADM_ID"].unique()).reset_index().copy()
Unique_id_dict.columns=["index","HADM_ID"]
Unique_id_dict["unique_id"]=Unique_id_dict["HADM_ID"].map(d)
Unique_id_dict.to_csv("unique_id_dict.csv")

unique_id_df = pd.read_csv("unique_id_dict.csv")
d = dict(zip(unique_id_df["HADM_ID"].values,unique_id_df["unique_id"].values))

In [18]:
admissions=pd.read_csv(file_path + "Admissions_processed.csv")
death_tags_s=admissions.groupby("HADM_ID")["DEATHTAG"].unique().astype(int).to_frame().reset_index()
death_tags_df=death_tags_s.loc[death_tags_s["HADM_ID"].isin(complete_df["HADM_ID"])].copy()
death_tags_df["unique_id"]=death_tags_df["HADM_ID"].map(d)
death_tags_df.sort_values(by="unique_id",inplace=True)
death_tags_df.rename(columns={"DEATHTAG":"Value"},inplace=True)
death_tags_df.set_index("unique_id", inplace=True)
death_tags_df.drop("HADM_ID", axis=1, inplace=True)
death_tags_df.to_csv("complete_death_tags.csv")

complete_df["unique_id"] = complete_df["HADM_ID"].map(d)

In [19]:
complete_df.rename(columns={"unique_id": "ID", "TIME_STAMP": "Time"}, inplace=True)
complete_df.drop(["HADM_ID"], axis=1, inplace=True)
complete_df.set_index(["ID"], inplace=True)
complete_df

,Time,Value_label_0,Mask_label_0,Value_label_1,Mask_label_1,Value_label_2,Mask_label_2,Value_label_3,Mask_label_3,Value_label_4,...,Value_label_92,Mask_label_92,Value_label_95,Mask_label_95,Value_label_94,Mask_label_94,Value_label_93,Mask_label_93,Value_label_91,Mask_label_91
ID,,,,,,,,,,,,,,,,,,,,,
12405,0 days 00:00:00,15.0,1,25.0,1,0.0,0,103.0,1,0.8,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
12405,0 days 02:20:00,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
12405,0 days 02:45:00,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
12405,0 days 03:29:00,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
12405,0 days 13:11:00,12.0,1,25.0,1,8.9,1,107.0,1,0.6,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17504,1 days 09:55:00,13.0,1,23.0,1,7.9,1,101.0,1,0.9,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
17504,1 days 18:00:00,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
17504,1 days 18:26:00,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0


In [20]:
complete_df.to_csv("full_dataset.csv")